# 数据预处理
读取csv文件并将其转换为可处理的tensor

创建一个人工数据集，并存储再csv（Comma-
Separated Values逗号分隔值）文件中

In [18]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')   # Alley是街道的意思
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

`with ... as`用法简介：*紧跟`with`后面的语句被求值后，返回对象的`__enter__()`方法被调用，这个方法的返回值将被赋值给as后面的变量。当`with`后面的代码块全部被执行完之后，将调用前面返回对象的`__exit__()`方法*

从创建的csv文件中加载原始数据集

In [19]:
import pandas as pd

data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


NaN意味着缺失的数据，典型的方法包括 **插值** 和 **删除**（某向量中有数据缺失，直接删除整个向量），下面我们考虑插值

对于有数值的部分，考虑使用平均值填充缺失值

In [20]:
# inputs为第1，2列，outputs为第3列
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean()) # 使用平均值填充NaN
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


对于inputs中类别值或离散值（非数值），我们将NaN视为一个类别
第2列（Alley）中，有Pave和NaN两类

In [21]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


现在，inputs和outputs中的所有条目都转换为了数值类型，下面可以转换为tensor格式了

In [22]:
import torch 

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))